# Finding explicitly a CM elliptic curve with small Galois image

### Authors:

Francesco Campagna and Riccardo Pengo

### Abstract:

This file accompanies Section 5 of our paper "How big is the image of a Galois representation attached to a CM elliptic curve?".

Our aim is to apply the algorithm provided by the proof of Theorem 5.11 of our previous paper: "[Entanglement in the family of division fields of elliptic curves with complex multiplication](https://arxiv.org/abs/2006.00883)" in order to obtain an elliptic curve $E$ defined over the Hilbert class field $H := \mathbb{Q}(i,\sqrt{5})$ of the imaginary quadratic field $K := \mathbb{Q}(\sqrt{-5})$, such that $\operatorname{End}_H(E) \cong \mathcal{O}_K := \mathbb{Z}[\sqrt{-5}]$ and the image of the adelic Galois representation $\rho_E \colon \operatorname{Gal}(\overline{H}/H) \to \operatorname{Aut}_{\mathcal{O}_K}(E_\text{tors}) \cong \widehat{\mathcal{O}}_K^\times$ has index $2$ inside $\widehat{\mathcal{O}}_K^\times$.

---
Initialize the fields and the $j$-invariant to be used
***

In [2]:
K.<D> = NumberField(x^2+5) # Imaginary quadratic field K
H.<i> = K.extension(x^2+1) # Hilbert class field H of K

In [3]:
hilbert_class_polynomial(-20).base_extend(H).factor()

(x + 282880*D*i - 632000) * (x - 282880*D*i - 632000)

In [4]:
j = 282880*D*i + 632000 # j-invariant

***
Create the elliptic curve $E$, and compute its conductor
***

In [5]:
E = EllipticCurve_from_j(j,False)
print(E)

Elliptic Curve defined by y^2 = x^3 + (-1071214510080*D*i-2395312128000)*x + (-901828270977187840*D*i-2016549312397312000) over Number Field in i with defining polynomial x^2 + 1 over its base field


In [6]:
E.conductor().absolute_norm().factor()

2^16 * 3^8 * 5^4 * 11^8 * 19^4

In [7]:
(E.conductor()/9).absolute_norm().factor()

2^16 * 5^4 * 11^8 * 19^4

***
Find the endomorphism $E \to E$ corresponding to multiplication by $D = \sqrt{-5}$.
***

In [8]:
E.isogenies_prime_degree(5)

[Isogeny of degree 5 from Elliptic Curve defined by y^2 = x^3 + (-1071214510080*D*i-2395312128000)*x + (-901828270977187840*D*i-2016549312397312000) over Number Field in i with defining polynomial x^2 + 1 over its base field to Elliptic Curve defined by y^2 = x^3 + (-161535/2*D*i+626175/2)*x + (-127642900*i-65378720*D) over Number Field in i with defining polynomial x^2 + 1 over its base field]

In [9]:
isog = E.isogenies_prime_degree(5)[0]

In [10]:
# Modify codomain by isomorphism, to get back to E
isog.set_post_isomorphism((isog.codomain()).isomorphisms(E)[1])

In [11]:
isog.is_endomorphism()

True

In [12]:
# Checking that this isogeny is indeed multiplication by D,  
# and not by -D, using its formal expansion
isog.formal(prec=5)

D*t + (-51418296483840*i + 22994996428800*D)*t^5 + (243493633163840716800*i - 108893662869454848000*D)*t^7 + O(t^8)

***
Verify that the roots of the quadratic factor of the $3$-division polynomial of $E$ generate the $3$-ray class field $H_{3} := H(\sqrt{-3})$ of $K$.
***

In [13]:
H3.<h3> = H.extension(x^2+3) # 3-ray class field of K

In [14]:
E.division_polynomial(3).factor()

(3) * (x + (266816*D + 59840)*i - 26048*D + 594880) * (x + (266816*D - 59840)*i + 26048*D + 594880) * (x^2 + (-533632*D*i - 1189760)*x - 1193205432320*D*i - 2668089262080)

In [15]:
H3new.<h3new> = H.extension(E.division_polynomial(3).factor()[-1][0])

In [16]:
H3new.is_isomorphic(H3)

True

***
Find which prime ideal lying above $3$ corresponds to a given $x$-coordinate already lying in $H$.
***

In [17]:
# Pick a x-coordinate of 3-torsion points which lies in F
x3 = E.division_polynomial(3).factor()[0][0].roots()[0][0]
print(x3)

(-266816*D - 59840)*i + 26048*D - 594880


In [18]:
# Substitute x3 in the Weierstrass polynomial of E, 
# and get the corresponding quadratic extension L
R.<x,y,z> = PolynomialRing(H)
W = E.defining_polynomial() # Weierstrass equation
L.<l> = H.extension((W.subs({x:x3,y:x,z:1})).univariate_polynomial()) 
print(L)

Number Field in l with defining polynomial x^2 + (-7430172301066240*D - 32211081415884800)*i + 14405235575357440*D - 16614361504153600 over its base field


In [19]:
# Base-changing E to L
E1 = E.base_extend(L)

In [20]:
# Base-changing isog to E1
isog1 = E1.isogeny(isog.kernel_polynomial())
isog1.set_post_isomorphism((isog1.codomain()).isomorphisms(E1)[1])

In [21]:
# Check that isog1 is indeed an endomorphism
isog1.is_endomorphism()

True

In [22]:
# Check that isog1 is indeed multiplication by D
isog1.formal(prec=5)

D*t + (-51418296483840*i + 22994996428800*D)*t^5 + (243493633163840716800*i - 108893662869454848000*D)*t^7 + O(t^8)

In [23]:
# Let's find the primes of K lying above 3
K.factor(3)

(Fractional ideal (3, D + 1)) * (Fractional ideal (3, D + 2))

In [24]:
# The computation above shows that we only need to check whether
# it's D+1 or D-1 to kill the 3-torsion points of E defined over Fp3
T = E1.torsion_points()
for P in T:
    if (3*P == 0):
        print(isog1(P) + P == 0)

True
True
True


In [25]:
# Looking at the previous computation, we can define p3...
p3 = K.factor(3)[0][0]
print(p3)

Fractional ideal (3, D + 1)


***
Twist $E$ by $L$ to get a curve $E_0$ with $\mathcal{I}(E_0/F) = 2$
***

In [26]:
# Find generator alpha of L/H. 
# Recall that alpha is the square of the y-coordinates of [p3]-torsion points.
alpha = -(L.defining_polynomial())(0)
print(alpha)

(7430172301066240*D + 32211081415884800)*i - 14405235575357440*D + 16614361504153600


In [27]:
# Twist E by L
E0 = E.quadratic_twist(alpha)
print(E0)

Elliptic Curve defined by y^2 = x^3 + ((52207899179985268313321375501520714124492800000*D-164081968851382271841867180147636530105548800000)*i+73379687246738769527795236613492765601300480000*D+116740411528902587885128785521465763456614400000)*x + ((21761029610487216150878700516481273153405898873337187319502864384000000*D+6965444702618673582183451196371973176047292050344304524560171008000000)*i-3115041569714232306522593359440267692358065490281406261763244032000000*D+48659141469435185785433507129254555720355908378099412076259704832000000) over Number Field in i with defining polynomial x^2 + 1 over its base field


In [28]:
E0.base_extend(L).is_isomorphic(E.base_extend(L))

True

***
Compute the global minimal model and the conductor of $E_0$.
***

In [29]:
E0.global_minimal_model()

Elliptic Curve defined by y^2 + ((-1/2*D+1/2)*i-1/2*D-1/2)*x*y + ((-1/2*D-1/2)*i-1/2*D-1/2)*y = x^3 + x^2 + ((-D+2)*i)*x + (2*i-1) over Number Field in i with defining polynomial x^2 + 1 over its base field

In [30]:
E0.conductor().factor()

(Fractional ideal ((-1/2*D - 1/2)*i + 1/2*D + 1/2))^2

In [31]:
# This shows that the unique prime of K dividing 
# the conductor of E0 is p3
E0.conductor().relative_norm().factor()

(Fractional ideal (3, D + 1))^4

***
We observe that, in the extension $H \subseteq L = H(E[\mathfrak{p}_3])$, the unique prime ideal lying above $\overline{\mathfrak{p}}_3$ ramifies. This is possible since $9 \mid \mathfrak{f}_E$.
***

In [32]:
L.relative_discriminant().factor()

(Fractional ideal (-i + 1))^4 * (Fractional ideal ((-1/2*D + 1/2)*i + 1/2*D - 1/2)) * (Fractional ideal ((-1/2*D + 1)*i + 1/2)) * (Fractional ideal ((-1/2*D - 1)*i + 1/2)) * (Fractional ideal (3/2*D*i + 1/2)) * (Fractional ideal (-3/2*D*i + 1/2)) * (Fractional ideal (2*D*i + 1))

In [33]:
L.relative_discriminant().factor()[1][0].relative_norm().factor()

(Fractional ideal (3, D + 2))^2

***
Find the $3$-division fields of $E$ and $E_0$
***

In [34]:
# Convert the curves over a relative number field to curves over an absolute one
Habs.<habs> = H.absolute_field()
Eabs = E.base_extend(H.embeddings(Habs)[0])
E0abs = E0.base_extend(H.embeddings(Habs)[0])

In [35]:
# Find the 3-division field M = H(E[3])
M.<m> = Eabs.division_field(3)
print(M)

Number Field in m with defining polynomial x^16 + 9518080*x^15 - 132902297201541072*x^14 - 948455618065553906176000*x^13 + 14923398908018851986061413115495328*x^12 + 70926071870145768872946484376824449105920*x^11 - 697890466347008267369280072226549229863847649598464*x^10 - 1653553339692471636557401334391211307992188017544798208000*x^9 + 27602633897802378896379167035539388230203642539283374912580878385920*x^8 - 384003304131281023502049488020715836615744658371259961915960797626368000*x^7 + 978557960084328457568679237639851526560674899752843637007595590251838439571456*x^6 - 10191943589083787563169552644856812038233253607480478039437188745458614509759365120*x^5 + 14148176580048973006850272784628648955682591040879400518484497378878208646698392805941248*x^4 - 98176961853918208928166678454739084439342519178231237687862449323812546338703363216926310400*x^3 + 97062191388705158294015808757751838309602765995404939295420735499765825503970248073859334930628608*x^2 - 33682646523266536079198657878391

In [36]:
# Find the 3-division field M0 = H(E0[3])
M0.<m0> = E0abs.division_field(3)
print(M0)

Number Field in m0 with defining polynomial x^8 - 633471860319914976870400*x^7 + 997544739529840399169021468339704072482324480000*x^6 + 284182789615514719740526716345678555426831493879572590995737411584000000*x^5 + 201252318236281341511491334349455439129651749250365631020008365883550548174559351983308800000000*x^4 - 49523867259084687154256079860493804252788586402771555168917745079164454896345883922487468242994462720000000000*x^3 + 3559792723150733351073963118632594905582180828887254962835704227024649380965880634523179304482296171454678433792000000000000*x^2 + 63131814358307916945761347337113990877343742514273297886697774022068668213322091777053505767107038899422420859719818988748800000000000000*x + 327045615826612168057016536130937608967789786723540504133464141358484898545714014480723202815341082626393695049574302827066808871157760000000000000000


In [37]:
M0.is_isomorphic(H3)

True

***
Just to be sure, we check that $L$ is not an abelian extension of $K$.
***

In [38]:
Lrel.<lrel> = L.relativize(K.embeddings(L)[0])
Lrel.is_galois_relative()

False